In [3]:
#je fais mes import
from datetime import datetime
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime
import html5lib
import re
import os
#date=strftime("%Y-%m-%d")
#avec mysql.connector
#Nettoyage
def clean_nbs(f):
    f = float(re.sub('\xa0','',f).replace(",","."))
    return f
def Indice() :
    import mysql.connector
    
    # mydb = mysql.connector.connect(
    #   host="localhost",
    #   user="root",
    #   passwd="1996A1996a/",
    #   database="BOURSE"
    # )
    # print(mydb)
    
    #je m'occupe de chaque action de la bourse 
    url='https://investir.lesechos.fr/actions/cotations/cours-az-cac-40.html'
    r = requests.get(url)
    print(url, r.status_code)
    soup_ACT = BeautifulSoup(r.content,'html.parser')
    #Je prend tt les href ACTION
    href_action = []
    for elem in soup_ACT.find_all('a'):
        if "cours/action" in elem.get("href") and "investir.lesechos" not in elem.get("href") :
            href_action.append("https://investir.lesechos.fr"+elem.get("href"))
    len(href_action)
    #POUR CHAQUE ACTION JE PREND LES ELEMENT
    for elem in href_action:
        #time.sleep(random.uniform(1.0, 1.5))
        url= elem
        r = requests.get(url)
        print(url, r.status_code)#200 == Good
        soup_hist = BeautifulSoup(r.content,'html.parser')

        if r.status_code != 200:
            print(f"problème l'url suivante{r.status_code,url}")
            continue

        Dernier_A = []
        Var_A = []
        Haut_A = []
        Bas_A = []
        Open_A = []
        Volume_A = []
        ISIN = []

        for elem in soup_hist.find_all('div',attrs={"class":"val-cours"}):
            Val = (re.findall("\d+.",elem.span.text.strip()))
            join = "".join(Val)
            elemClean = clean_nbs(join)
            Dernier_A.append(elemClean)

        for elem in soup_hist.find_all('div',attrs={"data-field":"variationHeaderFiche"}):
            elV = clean_nbs(elem.text.strip().replace("%",""))
            Var_A.append(elV)

        for elem in soup_hist.find_all('span',attrs={"data-field":"high"}):
            Val = (re.findall("\d+.",elem.text.strip()))
            join = "".join(Val)
            elemClean = clean_nbs(join)
            Haut_A.append(elemClean)

        for elem in soup_hist.find_all('span',attrs={"data-field":"low"}):
            Val = (re.findall("\d+.",elem.text.strip()))
            join = "".join(Val)
            elemClean = clean_nbs(join)
            Bas_A.append(elemClean)

        for elem in soup_hist.find_all('span',attrs={"data-field":"open"}):
            Val = (re.findall("\d+.",elem.text.strip()))
            join = "".join(Val)
            elemClean = clean_nbs(join)
            Open_A.append(elemClean)

        for elem in soup_hist.find_all('span',attrs={"data-field":"volume"}):
            Val = (re.findall("\d+.",elem.text.strip()))
            join = "".join(Val)
            try :
                elemClean = clean_nbs(join)
                Volume_A.append(elemClean)
                print(type(Volume_A))
            except :
                #elemClean = float(re.sub('\xa0','',join).replace(",",""))
                #Volume_A.append(elemClean)
                Volume_A = float(0)
                print(type(Volume_A))

        for elem in soup_hist.find_all('div',attrs={"class":"info-valeur"}):
            ISIN.append(re.findall("(\w{2}\d+)", elem.h2.text.strip()))
            ISIN = ISIN[0]
            ISIN = "".join(ISIN)

        print(Volume_A) 
        #try :  
        df_ACT = pd.DataFrame({"Valeur" :(Dernier_A),"Variation" :(Var_A),
            "+Haut" :(Haut_A[0]),"+Petit":(Bas_A[0]) ,"Open":(Open_A[0]) ,"Volume_A":(Volume_A), "ISIN":(ISIN)})
    return df_ACT